In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
def weigths_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear): # data type
        nn.init.xavier_uniform(m.weight.data)
        nn.init.constant_(m.bias, 0.1)

In [3]:
def compute_accuracy(prob_cls, gt_cls):
    # return tensor
    prob_cls = torch.squeeze(prob_cls)
    gt_cls = torch.squeeze(gt_cls)

    # tensor 비교 >= true or false
    mask = torch.ge(gt_cls,0)
    valid_gt_cls = torch.masked_select(gt_cls, mask)
    valid_prob_cls = torch.masked_select(prob_cls,mask)

    #computer predicted accuracy
    size = min(valid_gt_cls.size()[0],valid_prob_cls.size()[0])
    prob_ones = torch.ge(valid_prob_cls, 0.6).float()
    right_ones = torch.eq(prob_ones, valid_gt_cls).float()

In [11]:
torch.ge(torch.tensor([[1, 1], [3, 4]]), torch.tensor([[1, 2], [4, 4]]))

tensor([[ True, False],
        [False,  True]])